In [1]:
# ATSIT 2019: dimensional speech emotion recognition from text feature

# uncomment these to run on CPU only
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
import matplotlib.pyplot as plt
import pickle
import keras.backend as K
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, LSTM, TimeDistributed, Bidirectional, Embedding,GRU, Dropout, Flatten, concatenate, Convolution1D, BatchNormalization
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from keras.layers.convolutional import Conv2D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
import codecs
from keras.optimizers import SGD, Adam, RMSprop
import io
import random as rn
import tensorflow as tf
from keras.layers.core import Dense, Activation
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss
import pickle
with open('../../dimensionalSM/IEMOCAP_full_release/data_collected_333.pickle', 'rb') as handle:
    data2 = pickle.load(handle)
    
x_train_mocap = []
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape
# remove outlier
# outlier = np.array([1674, 3427, 5086, 5093, 5096, 5104, 7821])
# mask = np.ones(len(vad), np.bool)
# mask[outlier] = 0
# vad = vad[mask]
# x_train_mocap = x_train_mocap[mask]


Using TensorFlow backend.


(10039, 200, 189, 1)

In [2]:
# split train/test
split = 8000

In [2]:
# API model, if use RNN, first two rnn layer must return_sequences=True
def api_model(a,b,c):
    
    input_mocap = Input(shape=(200,189,1))
    model_mocap0 = BatchNormalization()(input_mocap)
    model_mocap1 = Conv2D(64, 3, strides=(3, 3), padding="same")(model_mocap0)
    model_mocap2 = Dropout(0.2)(model_mocap1)
    model_mocap3 = Activation('relu')(model_mocap2)
    model_mocap4 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap3)
    model_mocap5 = Dropout(0.2)(model_mocap4)
    model_mocap6 = Activation('relu')(model_mocap5)
    model_mocap7 = Conv2D(256, 3,strides=(3, 3), padding="same")(model_mocap6)
    model_mocap8 = Dropout(0.2)(model_mocap7) 
    model_mocap9 = Activation('relu')(model_mocap8)
    model_mocap10 = Conv2D(128, 3,strides=(3, 3), padding="same")(model_mocap9)   
    model_mocap11 = Dropout(0.2)(model_mocap10)  
    model_mocap12 = Activation('relu')(model_mocap11)
    model_mocap13 = Conv2D(64, 3,strides=(3, 3), padding="same")(model_mocap12)  
    model_mocap14 = Dropout(0.2)(model_mocap13) 
    model_mocap15 = Activation('relu')(model_mocap14)
    model_mocap = Flatten()(model_mocap15) 
    #model_mocap = Dropout(dropout_rate)(model_mocap16)

    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_mocap) for name in target_names]

    model = Model(input_mocap, model_combined) 

    model.compile(loss=ccc_loss, 
                  loss_weights={'v': a, 'a': b, 'd': c},
                  optimizer='rmsprop', metrics=[ccc])
    return model

#def main(alpha, beta, gamma):
model = api_model(1, 1, 1)
#odel = api_model(0.1, 0.5, 0.4)
model.summary()
# main function for for LOSO


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 189, 1)  0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 200, 189, 1)  4           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 67, 63, 64)   640         batch_normalization_1[0][0]      
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 67, 63, 64)   0           conv2d_1[0][0]                   
___________

In [4]:
# #speech_to_npy
# model1 = api_model(1,1,1)
# earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
#                               restore_best_weights=True)
# hist1 = model1.fit(x_train_mocap[:8000], vad[:8000].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, 
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate(x_train_mocap[8000:], vad[8000:].T.tolist())
# print(eval_metrik1)

# # make prediction
# predict = model1.predict(x_train_mocap[6400:], batch_size=32)
# np.save('mocap_batch_npy8000', 
#          np.array(predict).reshape(3, 3639).T)


In [5]:
def main(time):
    model = api_model(1,1,1)
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model.fit( x_train_mocap[:8000], 
                      vad[:8000].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True, callbacks=[earlystop])
    metrik = model.evaluate( x_train_mocap[8000:], vad[8000:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model.predict(x_train_mocap[6400:], batch_size=32)
    np.save('npy8000/mocap_npy-8000batch'+str(time)+'.npy',  
             np.array(predict).reshape(3, 3639).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.1, 0.5, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3:], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('npy8000/mocap_2DCNN8000_batch-1.csv', header=['time', 'best', 'ave'])


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 53s 8ms/step - loss: 2.7523 - v_loss: 0.8747 - a_loss: 0.9330 - d_loss: 0.9445 - v_ccc: 0.1253 - a_ccc: 0.0670 - d_ccc: 0.0555 - val_loss: 2.6265 - val_v_loss: 0.9038 - val_a_loss: 0.8487 - val_d_loss: 0.8740 - val_v_ccc: 0.0962 - val_a_ccc: 0.1513 - val_d_ccc: 0.1260
Epoch 2/50
6400/6400 [==============================] - 50s 8ms/step - loss: 2.0784 - v_loss: 0.5605 - a_loss: 0.7487 - d_loss: 0.7692 - v_ccc: 0.4395 - a_ccc: 0.2513 - d_ccc: 0.2308 - val_loss: 2.5912 - val_v_loss: 0.9116 - val_a_loss: 0.8212 - val_d_loss: 0.8584 - val_v_ccc: 0.0884 - val_a_ccc: 0.1788 - val_d_ccc: 0.1416
Epoch 3/50
6400/6400 [==============================] - 49s 8ms/step - loss: 1.8942 - v_loss: 0.4808 - a_loss: 0.7004 - d_loss: 0.7129 - v_ccc: 0.5192 - a_ccc: 0.2996 - d_ccc: 0.2871 - val_loss: 2.5195 - val_v_loss: 0.89

6400/6400 [==============================] - 42s 7ms/step - loss: 1.3455 - v_loss: 0.3484 - a_loss: 0.5018 - d_loss: 0.4953 - v_ccc: 0.6516 - a_ccc: 0.4982 - d_ccc: 0.5047 - val_loss: 2.2680 - val_v_loss: 0.7779 - val_a_loss: 0.7151 - val_d_loss: 0.7750 - val_v_ccc: 0.2221 - val_a_ccc: 0.2849 - val_d_ccc: 0.2250
Epoch 17/50
6400/6400 [==============================] - 42s 6ms/step - loss: 1.3538 - v_loss: 0.3470 - a_loss: 0.5063 - d_loss: 0.5006 - v_ccc: 0.6530 - a_ccc: 0.4937 - d_ccc: 0.4994 - val_loss: 2.2560 - val_v_loss: 0.7762 - val_a_loss: 0.7014 - val_d_loss: 0.7784 - val_v_ccc: 0.2238 - val_a_ccc: 0.2986 - val_d_ccc: 0.2216
Epoch 18/50
6400/6400 [==============================] - 43s 7ms/step - loss: 1.3167 - v_loss: 0.3424 - a_loss: 0.4905 - d_loss: 0.4838 - v_ccc: 0.6576 - a_ccc: 0.5095 - d_ccc: 0.5162 - val_loss: 2.2453 - val_v_loss: 0.7425 - val_a_loss: 0.7112 - val_d_loss: 0.7916 - val_v_ccc: 0.2575 - val_a_ccc: 0.2888 - val_d_ccc: 0.2084
Epoch 19/50
6400/6400 [===========

Epoch 20/50
6400/6400 [==============================] - 42s 7ms/step - loss: 1.3160 - v_loss: 0.3337 - a_loss: 0.4952 - d_loss: 0.4871 - v_ccc: 0.6663 - a_ccc: 0.5048 - d_ccc: 0.5129 - val_loss: 2.1818 - val_v_loss: 0.7410 - val_a_loss: 0.6894 - val_d_loss: 0.7514 - val_v_ccc: 0.2590 - val_a_ccc: 0.3106 - val_d_ccc: 0.2486
Epoch 21/50
6400/6400 [==============================] - 42s 7ms/step - loss: 1.3162 - v_loss: 0.3403 - a_loss: 0.4878 - d_loss: 0.4881 - v_ccc: 0.6597 - a_ccc: 0.5122 - d_ccc: 0.5119 - val_loss: 2.2340 - val_v_loss: 0.7543 - val_a_loss: 0.7011 - val_d_loss: 0.7786 - val_v_ccc: 0.2457 - val_a_ccc: 0.2989 - val_d_ccc: 0.2214
Epoch 22/50
6400/6400 [==============================] - 43s 7ms/step - loss: 1.3039 - v_loss: 0.3323 - a_loss: 0.4839 - d_loss: 0.4877 - v_ccc: 0.6677 - a_ccc: 0.5161 - d_ccc: 0.5123 - val_loss: 2.2121 - val_v_loss: 0.7611 - val_a_loss: 0.6910 - val_d_loss: 0.7600 - val_v_ccc: 0.2389 - val_a_ccc: 0.3090 - val_d_ccc: 0.2400
Epoch 23/50
6400/6400 

Epoch 5/50
6400/6400 [==============================] - 43s 7ms/step - loss: 1.7907 - v_loss: 0.4246 - a_loss: 0.6871 - d_loss: 0.6790 - v_ccc: 0.5754 - a_ccc: 0.3129 - d_ccc: 0.3210 - val_loss: 2.5151 - val_v_loss: 0.8363 - val_a_loss: 0.8011 - val_d_loss: 0.8777 - val_v_ccc: 0.1637 - val_a_ccc: 0.1989 - val_d_ccc: 0.1223
Epoch 6/50
6400/6400 [==============================] - 43s 7ms/step - loss: 1.7439 - v_loss: 0.4134 - a_loss: 0.6656 - d_loss: 0.6649 - v_ccc: 0.5866 - a_ccc: 0.3344 - d_ccc: 0.3351 - val_loss: 2.5571 - val_v_loss: 0.8600 - val_a_loss: 0.7817 - val_d_loss: 0.9154 - val_v_ccc: 0.1400 - val_a_ccc: 0.2183 - val_d_ccc: 0.0846
Epoch 7/50
6400/6400 [==============================] - 42s 7ms/step - loss: 1.7259 - v_loss: 0.4113 - a_loss: 0.6592 - d_loss: 0.6554 - v_ccc: 0.5887 - a_ccc: 0.3408 - d_ccc: 0.3446 - val_loss: 2.3433 - val_v_loss: 0.8149 - val_a_loss: 0.7350 - val_d_loss: 0.7933 - val_v_ccc: 0.1851 - val_a_ccc: 0.2650 - val_d_ccc: 0.2067
Epoch 8/50
6400/6400 [===

Epoch 5/50
6400/6400 [==============================] - 41s 6ms/step - loss: 1.7465 - v_loss: 0.4212 - a_loss: 0.6658 - d_loss: 0.6595 - v_ccc: 0.5788 - a_ccc: 0.3342 - d_ccc: 0.3405 - val_loss: 2.4895 - val_v_loss: 0.8945 - val_a_loss: 0.7738 - val_d_loss: 0.8212 - val_v_ccc: 0.1055 - val_a_ccc: 0.2262 - val_d_ccc: 0.1788
Epoch 6/50
6400/6400 [==============================] - 41s 6ms/step - loss: 1.6969 - v_loss: 0.4233 - a_loss: 0.6409 - d_loss: 0.6327 - v_ccc: 0.5767 - a_ccc: 0.3591 - d_ccc: 0.3673 - val_loss: 2.4953 - val_v_loss: 0.8598 - val_a_loss: 0.7635 - val_d_loss: 0.8720 - val_v_ccc: 0.1402 - val_a_ccc: 0.2365 - val_d_ccc: 0.1280
Epoch 7/50
6400/6400 [==============================] - 41s 6ms/step - loss: 1.6297 - v_loss: 0.4115 - a_loss: 0.6147 - d_loss: 0.6036 - v_ccc: 0.5885 - a_ccc: 0.3853 - d_ccc: 0.3964 - val_loss: 2.3591 - val_v_loss: 0.7860 - val_a_loss: 0.7580 - val_d_loss: 0.8152 - val_v_ccc: 0.2140 - val_a_ccc: 0.2420 - val_d_ccc: 0.1848
Epoch 8/50
6400/6400 [===

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6400/6400 [==============================] - 51s 8ms/step - loss: 1.2766 - v_loss: 0.3339 - a_loss: 0.4758 - d_loss: 0.4669 - v_ccc: 0.6661 - a_ccc: 0.5242 - d_ccc: 0.5331 - val_loss: 2.2411 - val_v_loss: 0.7482 - val_a_loss: 0.7154 - val_d_loss: 0.7775 - val_v_ccc: 0.2518 - val_a_ccc: 0.2846 - val_d_ccc: 0.2225
Epoch 27/50
6400/6400 [==============================] - 50s 8ms/step - loss: 1.2751 - v_loss: 0.3327 - a_loss: 0.4723 - d_loss: 0.4700 - v_ccc: 0.6673 - a_ccc: 0.5277 - d_ccc: 0.5300 - val_loss: 2.2333 - val_v_loss: 0.7568 - val_a_loss: 0.7108 - val_d_loss: 0.7657 - val_v_ccc: 0.2432 - val_a_ccc: 0.2892 - val_d_ccc: 0.2343
Epoch 28/50
6400/6400 [==============================] - 50s 8ms/step - loss: 1.2672 - v_loss: 0.3309 - a_loss: 0.4731 - d_loss: 0.4632 - v_ccc: 0.6691 - a_ccc: 0.5269 - d_ccc: 0.5368 - val_loss: 2.2561 - val_v_loss: 0.7714 - val_a_loss: 0.6994 - val_d_loss: 0.7853 - val_v_ccc: 0.2286 - val_a_ccc: 0.3006 - val_d_ccc: 0.2147
Epoch 29/50
6400/6400 [===========

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6400/6400 [==============================] - 51s 8ms/step - loss: 1.3356 - v_loss: 0.3418 - a_loss: 0.4943 - d_loss: 0.4996 - v_ccc: 0.6582 - a_ccc: 0.5057 - d_ccc: 0.5004 - val_loss: 2.2403 - val_v_loss: 0.7207 - val_a_loss: 0.7474 - val_d_loss: 0.7723 - val_v_ccc: 0.2793 - val_a_ccc: 0.2526 - val_d_ccc: 0.2277
Epoch 18/50
6400/6400 [==============================] - 50s 8ms/step - loss: 1.3161 - v_loss: 0.3340 - a_loss: 0.4892 - d_loss: 0.4929 - v_ccc: 0.6660 - a_ccc: 0.5108 - d_ccc: 0.5071 - val_loss: 2.2486 - val_v_loss: 0.7454 - val_a_loss: 0.7286 - val_d_loss: 0.7747 - val_v_ccc: 0.2546 - val_a_ccc: 0.2714 - val_d_ccc: 0.2253
Epoch 19/50
6400/6400 [==============================] - 50s 8ms/step - loss: 1.3074 - v_loss: 0.3336 - a_loss: 0.4925 - d_loss: 0.4813 - v_ccc: 0.6664 - a_ccc: 0.5075 - d_ccc: 0.5187 - val_loss: 2.1812 - val_v_loss: 0.7525 - val_a_loss: 0.6864 - val_d_loss: 0.7423 - val_v_ccc: 0.2475 - val_a_ccc: 0.3136 - val_d_ccc: 0.2577
Epoch 20/50
6400/6400 [===========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

